<a href="https://colab.research.google.com/github/myazann/Lung_Cancer/blob/main/LungCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git init
!git pull https://github.com/myazann/TCIA-API-SDK.git

!cp -r /content/tcia-rest-client-python/src/* /content

!rm -rf tcia-rest-client-java/
!rm -rf tcia-rest-client-python/

!cp /content/drive/MyDrive/Annotation.zip /content
!unzip Annotation.zip
!rm Annotation.zip

!pip install xmltodict

from tciaclient import TCIAClient
import json
import os
import numpy as np
from xml.etree import cElementTree as ElementTree
import xmltodict
import urllib.request, urllib.error, urllib.parse

In [2]:
folders = np.array(os.listdir("Annotation"))
##folders = [i for i in folders if "A" in i]

bboxs = {}

for folder in folders:
  path = "Annotation/" + folder
  for img in os.listdir(path):
    file_path = path + "/" + img
    try:
      dt = xmltodict.parse(ElementTree.tostring(ElementTree.parse(file_path).getroot()))["annotation"]["object"]
      if isinstance(dt, list):
        box_list = []
        for box in dt:
          box_list.append(box)
      else:
        dt = dt["bndbox"]
      img_name = folder + "-" + img.split(".x")[0]
      bboxs[img_name] = dt
    except Exception as e:
      print(e)

not well-formed (invalid token): line 1, column 0


In [ ]:
tc = TCIAClient(baseUrl="https://services.cancerimagingarchive.net/services/v4", resource = "TCIA")
clctn = "Lung-PET-CT-Dx"
series = json.loads(tc.get_series(collection = clctn, modality = "CT").read())

ptnt_to_series_ids = {}

for elem in series:
  try:
    if elem["BodyPartExamined"] == "CHEST":
      pt_id = elem["PatientID"].split("-")[1]
      ptnt_to_series_ids[elem["SeriesInstanceUID"]] = pt_id
  except Exception as e:
    print(e)

sop_to_series_ids = {}

for series in ptnt_to_series_ids.keys():
  sop = json.loads(tc.get_SOP_instance(series).read())
  for id in sop:
    sop_to_series_ids[id["SOPInstanceUID"]] = series

imgs_with_bboxs = [box.split("-")[1] for box in bboxs.keys()]
sop_to_series_ids = dict((img, sop_to_series_ids[img]) for img in sop_to_series_ids.keys() if img in imgs_with_bboxs)

with open('ptnt_to_series_ids.json', 'w') as json_file:
  json.dump(ptnt_to_series_ids, json_file)

with open('sop_to_series_ids.json', 'w') as json_file:
  json.dump(sop_to_series_ids, json_file)

In [60]:
with open('ptnt_to_series_ids.json') as f:
  ptnt_to_series_ids = json.load(f)

with open('sop_to_series_ids.json') as f:
  sop_to_series_ids = json.load(f)

In [ ]:
if not os.path.exists("images"):
  os.mkdir("images")

existing_images = os.listdir("images")

for img in sop_to_series_ids.keys():
  download_path = "./images/" + img + ".dcm"
  if img not in existing_images:
    tc.get_single_image(sop_to_series_ids[img], img, downloadPath = download_path)
  print("zaten var")